In [1]:
import os
from dotenv import load_dotenv
from google import genai

load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
MODEL = "gemini-2.5-flash"


In [2]:
def triage_email(email_text: str) -> str:
    prompt = f"""
    Classify this email into ONE category:
    - ignore
    - notify_human
    - respond/act

    Email:
    {email_text}

    Return only one of: ignore, notify_human, respond/act.
    """

    response = client.models.generate_content(
        model=MODEL,
        contents=prompt
    )

    raw = response.text.strip().lower()

    if "ignore" in raw:
        return "ignore"
    if "notify" in raw:
        return "notify_human"
    return "respond/act"


In [18]:
def triage_email(email_text: str) -> str:
    text = email_text.lower()

    # 🔒 HARD RULES (no LLM needed)
    spam_keywords = ["otp", "verification code", "unsubscribe", "win", "offer", "free", "discount"]
    if any(word in text for word in spam_keywords):
        return "ignore"

    action_keywords = ["schedule", "meeting", "call", "discuss", "available", "reply", "confirm"]
    if any(word in text for word in action_keywords):
        return "respond/act"

    # 🧠 FALLBACK TO GEMINI (AMBIGUOUS CASES ONLY)
    prompt = f"""
You are an email triage system.

STRICT RULES:
- ignore → spam, promotions, OTPs, newsletters
- respond/act → direct requests requiring a reply or action
- notify_human → informational or unclear emails needing human judgment

EXAMPLES:
Email: "Your OTP is 123456"
Answer: ignore

Email: "Can we meet tomorrow at 3pm?"
Answer: respond/act

Email: "Please review this document when you have time"
Answer: notify_human

NOW CLASSIFY THIS EMAIL.

Email:
{email_text}

Return ONLY one of:
ignore
notify_human
respond/act
"""

    response = client.models.generate_content(
        model=MODEL,
        contents=prompt
    )

    raw = response.text.strip().lower()

    if raw in ["ignore", "notify_human", "respond/act"]:
        return raw

    # 🛑 FINAL SAFETY FALLBACK
    return "notify_human"


In [19]:
email = "Hi, can we schedule a call tomorrow at 3pm?"
triage_email(email)


'respond/act'

In [20]:
def route_decision(email_text: str):
    decision = triage_email(email_text)
    print("Triage decision:", decision)

    if decision == "ignore":
        print("Email ignored.")
        return

    if decision == "notify_human":
        print("Notify human: manual review required.")
        return

    if decision == "respond/act":
        print("Agent will generate a response.")


In [21]:
email = "Hi, can we schedule a call tomorrow at 3pm?"
route_decision(email)


Triage decision: respond/act
Agent will generate a response.


In [22]:
def generate_email_reply(email_text: str) -> str:
    prompt = f"""
    You are an email assistant.
    Write a polite and professional reply to the following email.

    Email:
    {email_text}
    """

    response = client.models.generate_content(
        model=MODEL,
        contents=prompt
    )

    return response.text.strip()


In [24]:
def route_decision(email_text: str):
    decision = triage_email(email_text)
    print("Triage decision:", decision)

    if decision == "ignore":
        print("Email ignored.")
        return

    if decision == "notify_human":
        print("Notify human: manual review required.")
        return

    if decision == "respond/act":
        print("Generating email draft...\n")
        reply = generate_email_reply(email_text)

        print("----- DRAFT EMAIL -----")
        print(reply)
        print("-----------------------")


In [25]:
email = "Hi, can we schedule a call tomorrow at 3pm?"
route_decision(email)


Triage decision: respond/act
Generating email draft...

----- DRAFT EMAIL -----
Subject: Re: Call Scheduling

Hi [Sender's Name, if known - otherwise "Hi there"],

Thank you for reaching out.

Yes, 3pm tomorrow works for me. Could you please confirm the timezone you're referring to?

Once confirmed, I'll be happy to send a calendar invite.

Looking forward to our call.

Best regards,

[Your Name]
-----------------------


In [26]:
golden_emails = [
    {
        "email": "Win a free iPhone now, click here!",
        "label": "ignore"
    },
    {
        "email": "Can we schedule a meeting tomorrow at 3pm?",
        "label": "respond/act"
    },
    {
        "email": "Please review this document when you have time.",
        "label": "notify_human"
    },
    {
        "email": "Your OTP is 123456",
        "label": "ignore"
    },
    {
        "email": "Are you available for a quick call today?",
        "label": "respond/act"
    }
]


In [34]:
 # only first 5 for now


In [28]:
import time

results = []

for idx, item in enumerate(golden_emails, start=1):
    email_text = item["email"]
    expected = item["label"]

    print(f"Running {idx}/{len(golden_emails)}...")
    
    predicted = triage_email(email_text)

    results.append({
        "email": email_text,
        "expected": expected,
        "predicted": predicted
    })

    time.sleep(8)  # safe for free tier


Running 1/5...
Running 2/5...
Running 3/5...
Running 4/5...
Running 5/5...


In [29]:
results


[{'email': 'Win a free iPhone now, click here!',
  'expected': 'ignore',
  'predicted': 'ignore'},
 {'email': 'Can we schedule a meeting tomorrow at 3pm?',
  'expected': 'respond/act',
  'predicted': 'respond/act'},
 {'email': 'Please review this document when you have time.',
  'expected': 'notify_human',
  'predicted': 'notify_human'},
 {'email': 'Your OTP is 123456', 'expected': 'ignore', 'predicted': 'ignore'},
 {'email': 'Are you available for a quick call today?',
  'expected': 'respond/act',
  'predicted': 'respond/act'}]

In [31]:
for r in results:
    if r["expected"] != r["predicted"]:
        print("❌ MISCLASSIFIED")
        print("Email:", r["email"])
        print("Expected:", r["expected"])
        print("Predicted:", r["predicted"])
        print()



In [33]:
correct = sum(
    1 for r in results if r["expected"] == r["predicted"]
)

accuracy = correct / len(results)
accuracy*100


100.0

Day 2 extract the email to the gmail box 

In [1]:
!pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2


In [2]:
import os
import base64
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
